In [12]:
import inference
import IPython.display
import numpy as np
from scipy.io import wavfile

In [2]:
model = inference.load_model()
audio_file_path = "data/audio1.wav"
text_p = 'A man talking as insects buzz by.'
audio_file_path_p = None
text_n = 'Plastic crackling as a bird is singing and chirping.'
audio_file_path_n = None
sr, pred = inference.inference(model, audio_file_path, text_p, audio_file_path_p, text_n, audio_file_path_n)

/home/vscode/.local/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/vscode/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

Load the specified checkpoint inference_model/music_audioset_epoch_15_esc_90.14.pt from users.
Load Checkpoint...


/home/vscode/.local/lib/python3.10/site-packages/laion_clap/clap_module/factory.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path,

logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm2.bias 	 Loaded
audio_branch.layers.0.blocks.0

/workspaces/CLAPSep/inference.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_checkpoint, map_location=device)


Separate audio from [data/audio1.wav] with textual query p: [A man talking as insects buzz by.] and n: [Plastic crackling as a bird is singing and chirping.]


In [3]:
IPython.display.Audio(audio_file_path)

In [4]:
IPython.display.Audio(pred, rate=sr)


In [5]:
# ライブラリの再読み込み
import importlib
importlib.reload(inference)

<module 'inference' from '/workspaces/CLAPSep/inference.py'>

In [15]:
def mix_signals(signal1, signal2, snr_db):
    # Calculate the power of the signals
    signal1_power = np.mean(signal1**2)
    signal2_power = np.mean(signal2**2)
    
    # Calculate the power of the noise
    snr_linear = 10**(snr_db / 10)
    noise_power = signal1_power / snr_linear
    
    # Calculate the scaling factor for the second signal
    scaling_factor = np.sqrt(noise_power / signal2_power)
    
    # Scale the second signal and mix it with the first signal
    signal2_scaled = scaling_factor * signal2
    mixed_signal = signal1 + signal2_scaled
    
    return mixed_signal

# Read the audio files
sample_rate1, signal1 = wavfile.read('./data/1100010001_ToyCar_case1_normal_IND_ch1_0001-2.wav')
sample_rate2, signal2 = wavfile.read('./data/1xxx140064_ToyCar_caseAll_noise_CNT_ch4_0064.wav')

# Ensure both signals have the same length
if len(signal1) > len(signal2):
    signal1 = signal1[:len(signal2)]
else:
    signal2 = signal2[:len(signal1)]

# Define the SNR range (5 dB steps)
snr_range = np.arange(15, -16, -5)  # From 0 dB to -30 dB with 5 dB steps

for snr_db in snr_range:
    # Mix the signals with the current SNR
    mixed_signal = mix_signals(signal1, signal2, snr_db)
    
    # Write the mixed signal to a new file
    output_filename = f'./data/mixed_signal_{snr_db}dB.wav'
    wavfile.write(output_filename, sample_rate1, mixed_signal.astype(np.int16))
    print(f'Generated {output_filename} with SNR {snr_db} dB')


Generated ./data/mixed_signal_15dB.wav with SNR 15 dB
Generated ./data/mixed_signal_10dB.wav with SNR 10 dB
Generated ./data/mixed_signal_5dB.wav with SNR 5 dB
Generated ./data/mixed_signal_0dB.wav with SNR 0 dB
Generated ./data/mixed_signal_-5dB.wav with SNR -5 dB
Generated ./data/mixed_signal_-10dB.wav with SNR -10 dB
Generated ./data/mixed_signal_-15dB.wav with SNR -15 dB


/tmp/ipykernel_73390/940858575.py:20: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate1, signal1 = wavfile.read('./data/1100010001_ToyCar_case1_normal_IND_ch1_0001-2.wav')
/tmp/ipykernel_73390/940858575.py:21: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate2, signal2 = wavfile.read('./data/1xxx140064_ToyCar_caseAll_noise_CNT_ch4_0064.wav')


In [24]:
model = inference.load_model()

audio_file_path = "data/mixed_signal_-10dB.wav"
text_p = ''
audio_file_path_p = None#"data/1100010001_ToyCar_case1_normal_IND_ch1_0001-2.wav"
text_n = ''
audio_file_path_n = "data/1xxx140064_ToyCar_caseAll_noise_CNT_ch4_0064_2.wav"
sr, pred = inference.inference(model, audio_file_path, text_p, audio_file_path_p, text_n, audio_file_path_n)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load the specified checkpoint inference_model/music_audioset_epoch_15_esc_90.14.pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.

In [26]:
IPython.display.Audio(audio_file_path, normalize=False)


In [ ]:
# IPython.display.Audio(audio_file_path_p, normalize=False)


In [28]:
IPython.display.Audio(pred, rate=sr, normalize=False)